# SE_Project

## 과거 농산물 데이터를 분석해 미래 농산물의 가격을 예측하는 사이트 제작하기

In [2]:
# 기본 라이브러리 추가

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
# 2021년 ~ 2024년 농산물 가격 데이터 가져오기

df = pd.read_csv("collected_prices_2021_to_2024_all_items_markets_cumulative.csv")

In [4]:
df.head()

,SALEDATE,WHSALNAME,CMPNAME,LARGENAME_API,MIDNAME_API,SMALLNAME,AVGAMT,STD,TOTQTY,MY_TARGET_ITEM_NAME,MY_TARGET_LARGENAME,MY_TARGET_MIDNAME,WHSALCD
0,20210104,천안,천안농협(공),엽경채류,배추,기타배추,9100.000,1kg 상자,127.0,배추,엽경채류,배추,340101
1,20210104,천안,천안농협(공),엽경채류,배추,월동배추,3100.000,12kg 그물망,2700.0,배추,엽경채류,배추,340101
2,20210104,천안,천안농협(공),엽경채류,배추,쌈배추,7000.000,8kg 상자,32.0,배추,엽경채류,배추,340101
3,20210104,천안,천안농협(공),엽경채류,배추,쌈배추,2733.333,4kg 상자,180.0,배추,엽경채류,배추,340101
4,20210104,천안,천안농협(공),엽경채류,배추,월동배추,2650.000,12kg 그물망,1200.0,배추,엽경채류,배추,340101


In [5]:
df.shape

(758432, 13)

In [6]:
# STD로부터 각 항목의 무게 데이터를 추출하는 함수 선언

def extract_weight_from_STD(std_string):
    if pd.isna(std_string):
        return np.nan
    std_string = str(std_string).lower()

    kg_match = re.search(r'(\d+\.?\d*)\s*kg', std_string)
    if kg_match:
        return float(kg_match.group(1))

    g_match = re.search(r'(\d+\.?\d)\s*g', std_string)
    if g_match:
        return float(g_match.group(1)) / 1000
    return np.nan

df['STD_kg'] = df['STD'].apply(extract_weight_from_STD)

In [7]:
df['STD_kg']

0          1.0
1         12.0
2          8.0
3          4.0
4         12.0
          ... 
758427     2.0
758428     2.0
758429     2.0
758430     2.0
758431     4.0
Name: STD_kg, Length: 758432, dtype: float64

In [8]:
df['PRICE_PER_KG'] = df['AVGAMT'] / df['STD_kg']

df['PRICE_PER_KG'].replace([np.inf, -np.inf], np.nan, inplace = True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_25004\922420796.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['PRICE_PER_KG'].replace([np.inf, -np.inf], np.nan, inplace = True)


In [9]:
df['PRICE_PER_KG']

0          9100.000000
1           258.333333
2           875.000000
3           683.333250
4           220.833333
              ...     
758427    17250.000000
758428    12495.000000
758429    16774.500000
758430    15825.000000
758431     4425.000000
Name: PRICE_PER_KG, Length: 758432, dtype: float64

In [10]:
columns_to_drop = ['WHSALNAME', 'CMPNAME', 'WHSALCD', 'MY_TARGET_ITEM_NAME', 'MY_TARGET_LARGENAME', 'MY_TARGET_MIDNAME']

In [11]:
df.drop(columns = columns_to_drop, axis = 1, inplace = True, errors = 'ignore')

In [12]:
df.head()

,SALEDATE,LARGENAME_API,MIDNAME_API,SMALLNAME,AVGAMT,STD,TOTQTY,STD_kg,PRICE_PER_KG
0,20210104,엽경채류,배추,기타배추,9100.000,1kg 상자,127.0,1.0,9100.000000
1,20210104,엽경채류,배추,월동배추,3100.000,12kg 그물망,2700.0,12.0,258.333333
2,20210104,엽경채류,배추,쌈배추,7000.000,8kg 상자,32.0,8.0,875.000000
3,20210104,엽경채류,배추,쌈배추,2733.333,4kg 상자,180.0,4.0,683.333250
4,20210104,엽경채류,배추,월동배추,2650.000,12kg 그물망,1200.0,12.0,220.833333


In [13]:
columns_for_grouping = ['SALEDATE', 'MIDNAME_API', 'PRICE_PER_KG']

df_cleaned = df.dropna(subset = columns_for_grouping)

df_cleaned = df_cleaned.sort_values(by = 'SALEDATE')

grouping_keys = ['SALEDATE', 'MIDNAME_API']

df_daily_avg_price = df_cleaned.groupby(grouping_keys)['PRICE_PER_KG'].mean().reset_index()

df_daily_avg_price.head()

,SALEDATE,MIDNAME_API,PRICE_PER_KG
0,20210102,깻잎,12340.352983
1,20210102,대파,2163.333333
2,20210102,마늘,4182.761905
3,20210102,무,760.000000
4,20210102,배추,713.000000


In [14]:
df_daily_avg_price.tail()

,SALEDATE,MIDNAME_API,PRICE_PER_KG
8673,20241231,마늘,5366.154234
8674,20241231,무,1437.325704
8675,20241231,배추,2748.734634
8676,20241231,양파,1316.285738
8677,20241231,홍고추,10211.080747


In [15]:
pivot_column_name = 'MIDNAME_API'

try:
    dataframe = df_cleaned.pivot_table(index = 'SALEDATE',
                                      columns = pivot_column_name,
                                      values = 'PRICE_PER_KG',
                                      aggfunc = 'mean')
except KeyError as e:
    print(f"Check Column's name")
except Exception as e:
    print(f"Other error happened")

In [16]:
dataframe.columns.name = None
dataframe = dataframe.reset_index()

In [17]:
df = dataframe.round(2)

In [18]:
df

,SALEDATE,깻잎,대파,마늘,무,배추,양파,홍고추
0,20210102,12340.35,2163.33,4182.76,760.00,713.00,1282.48,3862.77
1,20210104,8773.32,2083.49,4427.22,708.50,1946.90,1381.92,3539.80
2,20210105,10148.78,2316.91,4731.48,996.28,1307.79,1396.43,3281.04
3,20210106,11108.13,2555.18,4420.99,1012.55,1942.60,1484.83,3440.97
4,20210107,11615.35,2740.90,4080.54,694.47,1872.97,1488.01,3018.87
...,...,...,...,...,...,...,...,...
1241,20241226,11099.88,1487.55,5217.14,1332.04,2548.80,1161.23,11351.61
1242,20241227,11454.85,1503.53,4674.20,1545.93,3642.66,1200.78,10328.21
1243,20241228,11325.36,1396.15,4317.21,1325.48,2941.62,1292.75,12691.50
1244,20241230,11842.47,1566.44,4904.38,2270.71,2287.54,1167.73,10164.39


In [19]:
column_order = [
    'SALEDATE',
    '배추',
    '무',
    '마늘',
    '양파',
    '대파',
    '홍고추',
    '깻잎'
]

df = df[column_order]

In [20]:
df

,SALEDATE,배추,무,마늘,양파,대파,홍고추,깻잎
0,20210102,713.00,760.00,4182.76,1282.48,2163.33,3862.77,12340.35
1,20210104,1946.90,708.50,4427.22,1381.92,2083.49,3539.80,8773.32
2,20210105,1307.79,996.28,4731.48,1396.43,2316.91,3281.04,10148.78
3,20210106,1942.60,1012.55,4420.99,1484.83,2555.18,3440.97,11108.13
4,20210107,1872.97,694.47,4080.54,1488.01,2740.90,3018.87,11615.35
...,...,...,...,...,...,...,...,...
1241,20241226,2548.80,1332.04,5217.14,1161.23,1487.55,11351.61,11099.88
1242,20241227,3642.66,1545.93,4674.20,1200.78,1503.53,10328.21,11454.85
1243,20241228,2941.62,1325.48,4317.21,1292.75,1396.15,12691.50,11325.36
1244,20241230,2287.54,2270.71,4904.38,1167.73,1566.44,10164.39,11842.47


In [21]:
kor_to_eng = {
    'SALEDATE': 'date',
    '배추': 'cabbage',
    '무': 'radish',
    '마늘': 'garlic',
    '양파': 'onion',
    '대파': 'daikon',
    '홍고추': 'cilantro',
    '깻잎': 'artichoke'
}

df.columns = df.columns.map(kor_to_eng)

In [22]:
df['date'] = df['date'].astype(str)

In [23]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

In [24]:
df

,date,cabbage,radish,garlic,onion,daikon,cilantro,artichoke
0,2021-01-02,713.00,760.00,4182.76,1282.48,2163.33,3862.77,12340.35
1,2021-01-04,1946.90,708.50,4427.22,1381.92,2083.49,3539.80,8773.32
2,2021-01-05,1307.79,996.28,4731.48,1396.43,2316.91,3281.04,10148.78
3,2021-01-06,1942.60,1012.55,4420.99,1484.83,2555.18,3440.97,11108.13
4,2021-01-07,1872.97,694.47,4080.54,1488.01,2740.90,3018.87,11615.35
...,...,...,...,...,...,...,...,...
1241,2024-12-26,2548.80,1332.04,5217.14,1161.23,1487.55,11351.61,11099.88
1242,2024-12-27,3642.66,1545.93,4674.20,1200.78,1503.53,10328.21,11454.85
1243,2024-12-28,2941.62,1325.48,4317.21,1292.75,1396.15,12691.50,11325.36
1244,2024-12-30,2287.54,2270.71,4904.38,1167.73,1566.44,10164.39,11842.47


In [25]:
df.to_csv('train_set.csv')